In [59]:
#!/usr/bin/python
import spidev
import RPi.GPIO as GPIO
import time

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

def StartUp():
    GPIO.setmode(GPIO.BCM)
    PRESET = 25
    IO4 = 26
    GPIO.setup(PRESET,GPIO.OUT)
    GPIO.setup(IO4,GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
    print "Reset 25 - Low 1s"
    GPIO.output(PRESET,GPIO.LOW)
    time.sleep(3)
    print "Reset 25 - High 0.2s"
    GPIO.output(PRESET,GPIO.HIGH)
    time.sleep(0.2)

    spi = spidev.SpiDev()
    spi.open(0,1) # CS2 - FPGA, on CE1 = IO4
    spi.mode = 0b00
    print "spi.cshigh is " + str(spi.cshigh)
    print "spi mode is " + str(spi.mode)
    spi.max_speed_hz = 2000000
    print "spi maxspeed is "+str(spi.max_speed_hz)+"hz"
    return spi

def TestSPI(spi,ncycles):
    i = 0
    while i < ncycles:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        time.sleep(0.5)
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
        time.sleep(0.5)  
        i = i+1

def LoopSPI(spi):
    while 1:
        WriteFPGA(spi,0xEB,0x01) # 0: single mode 1 continious mode
        WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode

def LoopAcq(spi):
    while 1:
        WriteFPGA(spi,0xEB,0x00) # Doing 1 shot 
        WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
        WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software
        time.sleep(0.1) # sleep 10ms
    
def ClearMem(spi):
    WriteFPGA(spi,0xEF,0x01) # To access memory

def ConfigSPI(spi):
    WriteFPGA(spi,0xE0,0x14) # set sEEPon
    WriteFPGA(spi,0xE1,0x00) # set sEEPoff MSB
    WriteFPGA(spi,0xE2,0x35) # set sEEPoff LSB
    WriteFPGA(spi,0xE3,0x00) # set sEEDelayACQ sEEDelayACQ MSB
    WriteFPGA(spi,0xE4,0x05) # set sEEDelayACQ EEDelayACQ LSB
    WriteFPGA(spi,0xE5,0x32) # set sEEPeriod MSB // durée de l'acquisition ? 
    WriteFPGA(spi,0xE6,0xC8) # set sEEPeriod LSB 0x32C8 acq (13000t * 10ns = 130us)
    WriteFPGA(spi,0xE7,0x01) # Period of one cycle MSB
    WriteFPGA(spi,0xE8,0x86) # Period of one cycle 15 to 8
    WriteFPGA(spi,0xE9,0xA0) # Period of one cycle LSB
    #WriteFPGA(spi,0xEA,0x00) # Software Trig : As to be clear by software
    WriteFPGA(spi,0xEB,0x00) # 0: single mode 1 continious mode
    WriteFPGA(spi,0xEC,0x11) # Voltage gain control: 0V to 1V
    WriteFPGA(spi,0xED,0x02) # Frequency of ADC acquisition / sEEADC_freq (3 = 16Msps, 1 = 32, 0 = 64, 2 = 21Msps)
    WriteFPGA(spi,0xEE,0xA0) # How many cycles in countinious mode
    print "Config FPGA done!"

def WriteFPGA(spi,adress,value):
    spi.xfer([0xAA] )
    spi.xfer([adress] )
    spi.xfer([value] )

In [50]:
Fech = 1/((0+1)*(1.0/64))
print "-> "+str( Fech ) + " Msps"
Nacq = 0x32C8/100 * Fech
print "-> "+str(Nacq) + ' samples' 

-> 64.0 Msps
-> 8320.0 samples


In [60]:
spi = StartUp()
ConfigSPI(spi)

Reset 25 - Low 1s
Reset 25 - High 0.2s
spi.cshigh is False
spi mode is 0
spi maxspeed is 2000000hz
Config FPGA done!


In [41]:
LoopSPI(spi)

KeyboardInterrupt: 

In [ ]:
LoopAcq(spi)

In [106]:
#WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software

In [107]:
#WriteFPGA(spi,0xEA,0x01) # Cleaning memory pointer

In [52]:
TestSPI(spi,3) # LED2 clignote 3x

In [61]:

WriteFPGA(spi,0xEB,0x00) # Doing 1 shot 
WriteFPGA(spi,0xEF,0x01) # Cleaning memory pointer
WriteFPGA(spi,0xEA,0x01) # Software Trig : As to be clear by software

time.sleep(0.2) # sleep 10ms

In [56]:
#print spi.xfer([0xCC] )[0]

In [62]:
spi.max_speed_hz = 80*100000
spi.mode = 0b00

A = []
nAcq = 2500*2
for i in range(nAcq+1):
    A.append ( spi.xfer([0x00] )[0] )  
a = np.asarray(A)
np.savetxt( "y-"+str(spi.mode)+"-"+str(nAcq)+"-"+str(spi.max_speed_hz)+".csv", a, delimiter=";")

In [25]:
spi.close() 